In [ ]:
from pydoc import importfile
import datetime
import openpyxl
import pathlib
import shutil

START_DATE = datetime.datetime(2025, 3, 1)
END_DATE = datetime.datetime(2025, 4, 30)
BENEFICIARY = "UMONS - NUMÉDIART"

CONVENTION_NUMBER = "SleepSense -  DIFST 2380067"
RESEARCHER_FIRST_NAME = "Vincent"
RESEARCHER_LAST_NAME = "Stragier"
RESEARCHER_JOB_TITLE = "assistant de recherche"
CONVENTION_OCCUPATION = 100  # %
HOURS_PER_DAY_MIN = 7.6  # h
HOURS_PER_DAY_MAX = 9  # h
TIME_INCREMENT = 0.5  # h
UNDER_MIN_PROBABILITY = 0.05
FILLING_DATE = datetime.datetime.now(tz=datetime.timezone.utc)

researcher_holidays = [
    datetime.datetime(2024, 10, 14),
    datetime.datetime(2025, 1, 6),
    datetime.datetime(2025, 1, 31),
    datetime.datetime(2025, 3, 6),
    datetime.datetime(2025, 3, 17),
    datetime.datetime(2025, 3, 27),
    datetime.datetime(2025, 3, 28),
]

# We don't care if it's in the morning or the afternoon
# as long as we know about it.
researcher_half_days = [
    datetime.datetime(2025, 2, 12),
    datetime.datetime(2025, 3, 26),
]

sick_days = [
    datetime.datetime(2025, 4, 7),
    datetime.datetime(2025, 4, 8),
    datetime.datetime(2025, 4, 9),
]

years = list(range(START_DATE.year, END_DATE.year + 1))


def get_start_date(year):
    return (
        START_DATE
        if year == START_DATE.year
        else datetime.datetime(year, 1, 1)
    )


def get_end_date(year):
    return (
        END_DATE if year == END_DATE.year else datetime.datetime(year, 12, 31)
    )


years_and_files_to_generate = [
    (
        year,
        f"{get_start_date(year).date()}_-_{get_end_date(year).date()}___{BENEFICIARY}___{CONVENTION_NUMBER}___{RESEARCHER_LAST_NAME}_{RESEARCHER_FIRST_NAME}.xlsx".replace(
            " ", "_"
        ),
    )
    for year in years
]

years_and_files_to_generate

[(2025,
  '2025-03-01_-_2025-04-30___UMONS_-_NUMÉDIART___SleepSense_-__DIFST_2380067___Stragier_Vincent.xlsx')]

In [ ]:
utils = importfile("utils.py")

if researcher_half_days is None:
    researcher_half_days = []

content_per_file = {}

for year, file_template in years_and_files_to_generate:
    start_date = datetime.datetime(year, 1, 1)
    if year == START_DATE.year:
        start_date = START_DATE

    end_date = datetime.datetime(year, 12, 31)
    if year == END_DATE.year:
        end_date = END_DATE

    lines_in_month = {}
    for month in range(start_date.month, end_date.month + 1):
        month_str = f"{month:02d}"
        number_of_days = utils.number_of_days_in_month(year, month)

        first_day = 1
        if year == start_date.year and month == start_date.month:
            first_day = start_date.day

        last_day = number_of_days
        if year == end_date.year and month == end_date.month:
            last_day = end_date.day

        number_of_working_days, number_of_half_days = (
            utils.get_number_of_working_days(
                year,
                month,
                first_day,
                last_day,
                researcher_holidays,
                half_days=researcher_half_days,
                sick_days=sick_days,
            )
        )

        print(
            f"Year: {year}, Month: {month_str}, Number of working days: {number_of_working_days}, Number of half days: {number_of_half_days}"
        )

        work_hours_per_day, work_hours_per_half_day = (
            utils.get_work_days_duration(
                number_of_working_days,
                number_of_half_days,
                HOURS_PER_DAY_MIN,
                HOURS_PER_DAY_MAX,
                CONVENTION_OCCUPATION / 100,
                UNDER_MIN_PROBABILITY,
            )
        )

        lines_in_month[month_str] = []
        for day in range(1, number_of_days + 1):
            current_date = datetime.datetime(year, month, day)
            description, day_in_weekend = utils.get_description(
                current_date,
                first_day,
                last_day,
                year,
                researcher_holidays,
                sick_days,
            )

            work_hours = ""
            if description == "":
                if current_date in researcher_half_days:
                    work_hours = work_hours_per_half_day.pop(0)

                else:
                    work_hours = work_hours_per_day.pop(0)

            line = {
                "day": day,
                "date": current_date.strftime("%d/%m/%Y"),
                "research_time": work_hours,
                "non_research_time": "",
                "description": description,
                "day_in_weekend": day_in_weekend,
            }
            lines_in_month[month_str].append(line)

    content_per_file[file_template] = {
        "year": year,
        "sheets_content": lines_in_month,
        "convention_beneficiary": BENEFICIARY,
        "convention_number": CONVENTION_NUMBER,
        "researcher_first_name": RESEARCHER_FIRST_NAME,
        "researcher_last_name": RESEARCHER_LAST_NAME,
        "researcher_job_title": RESEARCHER_JOB_TITLE,
        "convention_occupation": CONVENTION_OCCUPATION,
    }

Year: 2025, Month: 03, Number of working days: 16, Number of half days: 1
Year: 2025, Month: 04, Number of working days: 18, Number of half days: 0


In [ ]:
# Open the template file
template_file = pathlib.Path("templates/template_spw.xlsx")

for file_template, content in content_per_file.items():
    # Copy the template file (as a file).
    destination_file = pathlib.Path("generated_timesheets") / file_template
    destination_file.parent.mkdir(parents=True, exist_ok=True)

    if not destination_file.exists():
        shutil.copy(template_file, destination_file)

    destination_workbook = openpyxl.load_workbook(
        destination_file, read_only=False, rich_text=True
    )

    sheets_to_keep = list(content["sheets_content"].keys())
    for sheet in destination_workbook.sheetnames:
        if sheet not in sheets_to_keep:
            destination_workbook.remove(destination_workbook[sheet])
            continue

        sheet_content = content["sheets_content"][sheet]
        sheet = destination_workbook[sheet]

        sheet["C3"].value = content["convention_beneficiary"]
        sheet["C4"].value = content["convention_number"]
        sheet["C5"].value = (
            f"{content['researcher_first_name']} {content['researcher_last_name']}"
        )
        sheet["D6"].value = f"{content['convention_occupation']} %"

        sheet["H3"].value = f"{content['year']}"

        for index, row in enumerate(
            sheet.iter_rows(min_row=39, max_row=50, min_col=1, max_col=6),
            start=9,
        ):
            for cell in row:
                if cell.value == "Date:":
                    sheet[f"D{cell.row}"].value = FILLING_DATE.strftime(
                        "%d/%m/%Y"
                    )

                if str(cell.value).startswith('=C5&",'):
                    cell.value = f'=C5&", {content["researcher_job_title"]}"'

        # raise Exception("STOP")

        for index, line in enumerate(sheet_content, start=9):
            initial_font = sheet[f"F{index}"].font
            initial_content = sheet[f"F{index}"].value

            research_time = line["research_time"]
            if isinstance(research_time, (float, int)):
                research_time = round(research_time, 2)

            non_research_time = line["non_research_time"]
            if isinstance(non_research_time, (float, int)):
                non_research_time = round(non_research_time, 2)

            sheet[f"A{index}"].value = line["day"]
            sheet[f"B{index}"].value = line["date"]
            sheet[f"C{index}"].value = research_time
            sheet[f"D{index}"].value = non_research_time
            sheet[f"F{index}"].value = line["description"]

            if line["day_in_weekend"]:
                sheet[f"F{index}"].font = openpyxl.styles.Font(
                    name=initial_font.name,
                    size=initial_font.size,
                    color="FF0000",
                    bold=True,
                )
            elif line["description"] != "":
                sheet[f"F{index}"].font = openpyxl.styles.Font(
                    name=initial_font.name,
                    size=initial_font.size,
                    color="FF0000",
                    bold=False,
                    italic=True,
                )
            else:
                if initial_content is not None and initial_content != "":
                    sheet[f"F{index}"].value = initial_content
                sheet[f"F{index}"].font = openpyxl.styles.Font(
                    name=initial_font.name,
                    size=initial_font.size,
                    color="000000",
                    bold=False,
                )

    destination_workbook.save(destination_file)
    destination_workbook.close()